In [1]:
from vizwiz_api.vizwiz import VizWiz
import spacy
import numpy as np
import json

# nlp = spacy.load("en_core_web_sm")

In [12]:
annFile = './annotations/val.json'
output = './annotations/val_split.json'

# results_base = './results/results_base.json'
# results_gt = './results/results_gt.json'

In [13]:
vizwiz = VizWiz(annFile, ignore_rejected=True, ignore_precanned=True)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created! imgs = 7750, anns = 33145


In [7]:
img_id = 4961
for ann in vizwiz.dataset['annotations']:
    if ann['image_id'] == img_id:
        print(ann['caption'])

a pack of black and white pokemon trading cards
A package of black and white pokemon trading cards
Someone is holding a new package of Pokemon Cards that says Black & White with some type of a horse or zebra looking character on the front.
A few hands are holding a Pokemon game cartridge.
A person is holding a video game in their hand.


In [14]:
anns_split = []
anns_base = []
anns_gt = []
images = []

counts = {}
# keys = [' in front of ', ' on top of ', ' above ', ' accross ', ' behind ', ' next to ', ' on ']
keys = [' in the middle ', ' to the left ', ' to the right ', ' in front of ', ' on top of ', ' in the distance ', ' in the foreground ', \
        ' in the background ', ' adjacent to ', ' next to ', ' over ', ' above ', ' below ', ' under ', ' further ', ' beyond ', ' nearby ', \
        ' between ', ' alongside ', ' among ', ' beside ', ' beneath ', ' behind ', ' across ', ' on ']

In [15]:
len(keys)

25

In [16]:
for ann in vizwiz.dataset['annotations']:
    caption = ann['caption'].lower()
    ann['split'] = None
    id = ann['id']
    image_id = ann['image_id']
    
    for key in keys:
        if key in caption:
            cap_split = caption.split(key,1)
            if len(cap_split[1].split()) >=3 and len(cap_split[0].split()) >=3:
                counts[key] = counts.get(key,0) + 1
                ann['split'] = cap_split
                ann['key'] = key
                anns_base.append({"image_id": id, "caption": ann['split'][0] + ' ' + ann['split'][1]})
                break
    
    if not ann['split']:
        ann['split'] = [caption,'']
        ann['key'] = None
        anns_base.append({"image_id": id, "caption": caption})
        
    anns_gt.append({"image_id": id, "caption": caption})
    images.append({"file_name": None, "id": id})
    
    ann['image_id'] = id # ann['id']
    ann['image_id_true'] = image_id
                
    anns_split.append(ann)
print('finished!')

finished!


In [17]:
sum(counts.values())

9865

In [7]:
json.dump({'annotations':anns_split, 'images':images}, open(output,'w'))
# json.dump({'annotations':anns_split}, open(output,'w'))
print('wrote to ' + output)

wrote to ./annotations/train_split.json
